In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/p

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-07 00:08:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.50MB/s    in 0.2s    

2023-03-07 00:08:49 (5.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [5]:
# Get the number of rows in the DataFrame.
df.count()

1752932

# Transform the Data

## Create the "review_id_table".

In [23]:
from pyspark.sql.functions import to_date, col
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
# Select the necessary columns
review_id_df = df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))
# Cast "customer_id" and "product_parent" column to integer
review_id_df = review_id_df.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))
# Show the resulting DataFrame
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
| REV51EW323H8W|    9287389|B010UX9T5I|     446691106| 2015-08-31|
|R2GQ3W03WIUZKE|   32840762|B00VWBY7SC|     271204734| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select("product_id", "product_title").dropDuplicates()
# Show the resulting DataFrame
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Y8QMQN6|Best Baby Teether...|
|B002WN2BY6|The First Years T...|
|B000AV7O4O|Boon Frog Pod Bat...|
|B00UFOBDNY|Kinsa Smart Stick...|
|B00XF7DZ34|Critter Piller Ki...|
|B00WKIR22E|Poly Comfortable ...|
|B001EU35BQ|Peg Perego Tatami...|
|B001TWI81G|Thermos Reusable ...|
|B007EJ7J4U|Kaloo Doudou Beig...|
|B00M8MGGQA|Large Baby Bag Or...|
|B00Y2CJWKQ|Clinical Digital ...|
|B00Y8ION80|★ Baby Pacifier &...|
|B00UJOWZ96|Corner Cushion Ba...|
|B00DZ3K64Y|Sassy Soft Grip N...|
|B00JBYPKE2|Maxi Cosi Pria 85...|
|B00HK6GWTU|Ju-Ju-Be Be Light...|
|B00IYU4O8W|   Lamaze Cloth Book|
|B00SCDM4FQ|Olababy Silicone ...|
|B004UOTWJ2|Charlie Banana - ...|
|B001LF3YQK|Skip Hop Baby Swi...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [27]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))
# Show the resulting DataFrame
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50583551|             5|
|   31325270|             4|
|   24063505|             1|
|   14335758|             1|
|   43801871|             4|
|   48782599|             4|
|   15553793|             4|
|   51506042|             1|
|   30238476|             2|
|   23810736|             2|
|   24413027|             3|
|   22673914|             2|
|   11899607|             1|
|   12536186|             1|
|   15311746|             4|
|   26246534|             1|
|   38361105|             7|
|   23684208|             1|
|   13588258|             1|
|   47049654|             9|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [34]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))
# Show the resulting DataFrame
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
| R6RBP4HTE67SY|          5|            0|          0|   N|
|R15B3EU40RSU2W|          5|            0|          0|   N|
| RP4DD53A4ZJA2|          5|            0|          0|   N|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|
| REV51EW323H8W|          5|            0|          0|   N|
|R2GQ3W03WIUZKE|          5|            0|          0|   N|
| RTI1YI7K6GE3D|          5|            0|          0|   N|
|R3V9C2C0SPSZU6|          5|            

# Load

In [14]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [24]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [25]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [28]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [35]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)